### Template notebook for finetuning MaskDINO on different datasets.

Notebook to finetune MaskDINO to detect and segment object classes relevant for the Storing Groceries task at RoboCup@Home2023. To finetune for other tasks, just select relevant datasets and labels as exemplified here.

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

import torch
import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()

# import some common libraries
import numpy as np
import os, shutil, json, random, cv2
from datetime import datetime

# import some common detectron2 utilities
from detectron2.structures import BoxMode
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.projects.deeplab import add_deeplab_config
#from detectron2.data.datasets import register_coco_instances

import sys
sys.path.append('contrib/MaskDINO')

sys.path.append("..")
from helpers.util import get_timestamp, select_classes, merge_to_superclass, concat_datasets, make_archive
from helpers.fiftyone_detectron2_bridge import clean_instances, detectron_to_fo, get_fiftyone_dicts

In [2]:
experiment_name = 'example_detector'
model_type = 'swin' #'r50', 'swin' -> r50 uses fewer resources, swin is more performant

#### Dataset creation

Usually we want to add the base dataset from Bonn or the day 1 dataset from Bordeaux, the older robotics hall dataset at AIS and probably also at least one more dataset created in Bordeaux. For some tasks it might be beneficial to add some of the auxiliary image search datasets as well. Locally it can make sense to select a different validation set as well. The available datasets and the object classes they cover are listed in *dataset_overview.ipynb*.

In [3]:
# add the base dataset
# name for the dataset
name = "robocup23_day1_coco"

# The directory containing the dataset to import
data_dir = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/Bordeaux/bordeaux_day_1_coco"
labels_path = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/Bordeaux/bordeaux_day_1_coco/annotations"
train_dir = data_dir + '/train'
valid_dir = data_dir + '/val'
train_labels_path = labels_path + '/instances_Train.json'
valid_labels_path = labels_path + '/instances_Validation.json'

# setup the fiftyone dataset
dataset_type = fo.types.COCODetectionDataset

train_dataset = fo.Dataset.from_dir(
    dataset_type=dataset_type,
    data_path=train_dir,
    labels_path=train_labels_path,
    name=name+'_train',
)

valid_dataset = fo.Dataset.from_dir(
    dataset_type=dataset_type,
    data_path=valid_dir,
    labels_path=valid_labels_path,
    name=name+'_valid',
)

# most of the time we use all classes here and filter the total dataset at the end instead
relevant_classes = ['apple', 'bin', 'chair', 'chocolate_pudding_box', 'cleanser_bottle', 'coffee_can', 'couch', 
                    'couch_table', 'cupboard', 'dishwasher', 'door', 'dresser', 'fork', 'fridge', 'fridge_handle', 
                    'jacket_stand', 'knife', 'lamp', 'mustard_bottle', 'person', 'pringles', 'rubiks_cube', 'shelf', 
                    'sink', 'spoon', 'strawberry_gelatin_box', 'sugar_box', 'table', 'tv']

train_dataset = select_classes(train_dataset, labels=relevant_classes)
valid_dataset = select_classes(valid_dataset, labels=relevant_classes)

print('Found train dataset labels:')
print(train_dataset.distinct("segmentations.detections.label"))
print('Found valid dataset labels:')
print(valid_dataset.distinct("segmentations.detections.label"))

# merge training and validation data here (because at competition time we are greedy)
# we will create a validation set further down the line
combined_train_dataset = concat_datasets([train_dataset, valid_dataset])
valid_dataset = None

 100% |█████████████████| 100/100 [2.2s elapsed, 0s remaining, 44.6 samples/s]        
 100% |███████████████████| 25/25 [653.9ms elapsed, 0s remaining, 38.3 samples/s]      
Found train dataset labels:
['apple', 'bin', 'chair', 'chocolate_pudding_box', 'cleanser_bottle', 'coffee_can', 'couch', 'couch_table', 'cupboard', 'dishwasher', 'door', 'dresser', 'fork', 'fridge', 'fridge_handle', 'jacket_stand', 'knife', 'lamp', 'mustard_bottle', 'person', 'pringles', 'rubiks_cube', 'shelf', 'sink', 'spoon', 'strawberry_gelatin_box', 'sugar_box', 'table', 'tv']
Found valid dataset labels:
['apple', 'bin', 'chair', 'chocolate_pudding_box', 'cleanser_bottle', 'coffee_can', 'couch', 'couch_table', 'cupboard', 'dishwasher', 'door', 'fork', 'knife', 'lamp', 'mustard_bottle', 'person', 'pringles', 'rubiks_cube', 'shelf', 'sink', 'spoon', 'strawberry_gelatin_box', 'sugar_box', 'table', 'tv']


In [4]:
# we continue by adding the dataset containing furniture from the arenas.
# again use the validation set for training instead
combined_train_dataset = combined_train_dataset

USE_LOCAL_FURNITURE = True

if USE_LOCAL_FURNITURE:
    # add the older robotics hall dataset
    name = "robocup23_local_furniture"

    # The directory containing the dataset to import
    data_dir = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/Bordeaux/arena_furniture_coco"
    labels_path = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/Bordeaux/arena_furniture_coco/annotations"

    train_dir = data_dir + '/train'
    valid_dir = data_dir + '/val'
    train_labels_path = labels_path + '/instances_Train.json'
    valid_labels_path = labels_path + '/instances_Validation.json'

    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset  # for example

    train_dataset_add = fo.Dataset.from_dir(
        dataset_type=dataset_type,
        data_path=train_dir,
        labels_path=train_labels_path,
        name=name+'_train',
    )

    valid_dataset_add = fo.Dataset.from_dir(
        dataset_type=dataset_type,
        data_path=valid_dir,
        labels_path=valid_labels_path,
        name=name+'_valid',
    )

    print('Adding dataset ' + name + '...')
    print('Found train dataset labels:')
    print(train_dataset_add.distinct("segmentations.detections.label"))
    print('Found valid dataset labels:')
    print(valid_dataset_add.distinct("segmentations.detections.label"))

    # most of the time we use all classes here and filter the total dataset at the end instead
    relevant_classes = ['chair', 'couch', 'couch_table', 'cupboard', 'dishwasher', 'dishwasher_slot', 
                        'dishwasher_tray', 'door', 'door_handle', 'dresser', 'fridge', 'fridge_handle', 'jacket_stand', 
                        'lamp', 'person', 'shelf', 'sink', 'table', 'tv']

    train_dataset_add = select_classes(train_dataset_add, labels=relevant_classes)
    valid_dataset_add = select_classes(valid_dataset_add, labels=relevant_classes)

    combined_train_dataset = concat_datasets([combined_train_dataset, train_dataset_add])
    combined_train_dataset = concat_datasets([combined_train_dataset, valid_dataset_add])

 100% |█████████████████| 100/100 [1.3s elapsed, 0s remaining, 78.2 samples/s]         
 100% |███████████████████| 50/50 [611.2ms elapsed, 0s remaining, 81.8 samples/s]      
Adding dataset robocup23_local_furniture...
Found train dataset labels:
['bed', 'chair', 'couch', 'couch_table', 'cupboard', 'dishwasher', 'dishwasher_slot', 'dishwasher_tray', 'door', 'door_handle', 'dresser', 'fridge', 'fridge_handle', 'jacket_stand', 'lamp', 'microwave_oven', 'person', 'shelf', 'sink', 'table', 'tv']
Found valid dataset labels:
['bed', 'bin', 'chair', 'couch', 'couch_table', 'cupboard', 'dishwasher', 'dishwasher_slot', 'dishwasher_tray', 'door', 'door_handle', 'dresser', 'fridge', 'fridge_handle', 'jacket_stand', 'lamp', 'person', 'shelf', 'sink', 'table', 'tv']


In [5]:
# now we add our dataset from day 2 in Bordeaux
# at this point we actually initialize the validation set cleanly
combined_train_dataset = combined_train_dataset

USE_DAY2 = True
if USE_DAY2:
    # add the older robotics hall dataset
    name = "bordeaux_day2"

    # The directory containing the dataset to import
    data_dir = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/Bordeaux/bordeaux_day_2_coco"
    labels_path = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/Bordeaux/bordeaux_day_2_coco/annotations"

    train_dir = data_dir + '/train'
    valid_dir = data_dir + '/val'
    train_labels_path = labels_path + '/instances_Train.json'
    valid_labels_path = labels_path + '/instances_Validation.json'

    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset  # for example

    train_dataset_add = fo.Dataset.from_dir(
        dataset_type=dataset_type,
        data_path=train_dir,
        labels_path=train_labels_path,
        name=name+'_train',
    )
    
    valid_dataset_add = fo.Dataset.from_dir(
    dataset_type=dataset_type,
    data_path=valid_dir,
    labels_path=valid_labels_path,
    name=name+'_valid',
    )

    print('Adding dataset ' + name + '...')
    print('Found train dataset labels:')
    print(train_dataset_add.distinct("segmentations.detections.label"))
    print('Found valid dataset labels:')
    print(valid_dataset_add.distinct("segmentations.detections.label"))

    # most of the time we use all classes here and filter the total dataset at the end instead
    relevant_classes = ['apple', 'bag', 'banana', 'bed', 'bin', 'bowl', 'candle', 'cereals', 'chair',
                        'chocolate_pudding_box', 'cola', 'couch', 'couch_table', 'cracker_box', 'cupboard', 
                        'decoration', 'dishwasher', 'dishwasher_tray', 'door', 'door_handle', 'dresser', 'fridge', 
                        'iced_tea', 'jacket_stand', 'juice_pack', 'lamp', 'lemon', 'milk_bottle', 'mustard_bottle', 
                        'orange', 'orange_juice', 'peach', 'pear', 'person', 'plum', 'potted_meat_can', 'pringles', 
                        'red_wine', 'rubiks_cube', 'shelf', 'shelf_door', 'sink', 'soccer_ball', 'sponge', 'strawberry', 
                        'strawberry_gelatin_box', 'sugar_box', 'table', 'tennis_ball', 'tomato_soup_can', 'tray', 
                        'tropical_juice', 'tuna_fish_can', 'tv']

    train_dataset_add = select_classes(train_dataset_add, labels=relevant_classes)
    valid_dataset_add = select_classes(valid_dataset_add, labels=relevant_classes)

    combined_train_dataset = concat_datasets([combined_train_dataset, train_dataset_add])
    combined_valid_dataset = valid_dataset_add

 100% |█████████████████| 253/253 [5.2s elapsed, 0s remaining, 38.1 samples/s]      
 100% |███████████████████| 32/32 [572.5ms elapsed, 0s remaining, 55.9 samples/s]      
Adding dataset bordeaux_day2...
Found train dataset labels:
['apple', 'bag', 'banana', 'bed', 'bin', 'bowl', 'candle', 'cereals', 'chair', 'chocolate_pudding_box', 'cola', 'couch', 'couch_table', 'cracker_box', 'cupboard', 'decoration', 'dice', 'dishwasher', 'dishwasher_tray', 'door', 'door_handle', 'dresser', 'fridge', 'iced_tea', 'jacket_stand', 'juice_pack', 'lamp', 'lemon', 'milk_bottle', 'mustard_bottle', 'orange', 'orange_juice', 'peach', 'pear', 'person', 'plum', 'potted_meat_can', 'pringles', 'red_wine', 'rubiks_cube', 'shelf', 'shelf_door', 'sink', 'soccer_ball', 'sponge', 'strawberry', 'strawberry_gelatin_box', 'sugar_box', 'table', 'tennis_ball', 'tomato_soup_can', 'tray', 'tropical_juice', 'tuna_fish_can', 'tv']
Found valid dataset labels:
['banana', 'bed', 'bin', 'bowl', 'candle', 'cereals', 'chair', 'c

In [6]:
# now we add a rather large dataset from Bonn. 
# here the orange juice looks a bit like the tropical juice at robocup, so we just remap that class name
# the validation set is combined with the validation set from this particular dataset

USE_AIS_OBJECTS = True
if USE_AIS_OBJECTS:
    # add the older robotics hall dataset
    name = "robocup23_ais_coco"

    # The directory containing the dataset to import
    data_dir = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/Bonn/robocup23_ais_coco"
    labels_path = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/Bonn/robocup23_ais_coco/annotations"

    train_dir = data_dir + '/train'
    valid_dir = data_dir + '/val'
    train_labels_path = labels_path + '/instances_Train.json'
    valid_labels_path = labels_path + '/instances_Validation.json'

    dataset_type = fo.types.COCODetectionDataset  # for example

    train_dataset_add = fo.Dataset.from_dir(
        dataset_type=dataset_type,
        data_path=train_dir,
        labels_path=train_labels_path,
        name=name+'_train',
    )

    valid_dataset_add = fo.Dataset.from_dir(
        dataset_type=dataset_type,
        data_path=valid_dir,
        labels_path=valid_labels_path,
        name=name+'_valid',
    )

    print('Adding dataset ' + name + '...')
    print('Found train dataset labels:')
    print(train_dataset_add.distinct("segmentations.detections.label"))
    print('Found valid dataset labels:')
    print(valid_dataset_add.distinct("segmentations.detections.label"))
    


    # most of the time we use all classes here and filter the total dataset at the end instead
    relevant_classes = ['apple', 'bag', 'banana', 'baseball', 'bowl', 'cereals',
                        'chair', 'chocolate_pudding_box', 'cleanser_bottle', 'coffee_can', 'cracker_box', 'dice', 
                        'dishwasher_tab', 'fork', 'knife', 'lemon', 'milk_bottle', 'mug', 'mustard_bottle', 
                        'orange', 'orange_juice', 'peach', 'pear', 'person', 
                        'plate', 'plum', 'potted_meat_can', 'pringles', 'rubiks_cube', 'soccer_ball', 
                        'spoon', 'strawberry', 'strawberry_gelatin_box', 'sugar_box', 'table', 
                        'tennis_ball', 'tomato_soup_can', 'tuna_fish_can']

    train_dataset_add = select_classes(train_dataset_add, labels=relevant_classes)
    valid_dataset_add = select_classes(valid_dataset_add, labels=relevant_classes)
    
    old_class_labels = ['orange_juice']
    new_class_labels = ['tropical_juice']
    train_dataset_add = merge_to_superclass(train_dataset_add, new_class_labels, old_class_labels)
    valid_dataset_add = merge_to_superclass(valid_dataset_add, new_class_labels, old_class_labels)
    
    combined_train_dataset = concat_datasets([combined_train_dataset, train_dataset_add])
    combined_valid_dataset = concat_datasets([combined_valid_dataset, valid_dataset_add])

 100% |█████████████████| 300/300 [8.8s elapsed, 0s remaining, 35.2 samples/s]      
 100% |███████████████████| 30/30 [692.4ms elapsed, 0s remaining, 43.4 samples/s]      
Adding dataset robocup23_ais_coco...
Found train dataset labels:
['almdudler', 'apple', 'apple_juice', 'bag', 'banana', 'baseball', 'booster', 'bowl', 'cereals', 'chair', 'chocolate_pudding_box', 'cleanser_bottle', 'coffee_can', 'coke', 'cracker_box', 'dice', 'dishwasher_tab', 'fork', 'golf_ball', 'iso_drink', 'knife', 'lemon', 'milk_bottle', 'milk_carton', 'mug', 'mustard_bottle', 'orange', 'orange_juice', 'peach', 'pear', 'person', 'plate', 'plum', 'potted_meat_can', 'pringles', 'racquet_ball', 'rubiks_cube', 'soccer_ball', 'soft_ball', 'spoon', 'strawberry', 'strawberry_gelatin_box', 'sugar_box', 'table', 'tennis_ball', 'tomato_soup_can', 'tuna_fish_can']
Found valid dataset labels:
['almdudler', 'apple', 'apple_juice', 'bag', 'banana', 'baseball', 'booster', 'bowl', 'cereals', 'chair', 'chocolate_pudding_box', '

In [7]:
# this dataset contains some objects which we considered challenging after initial tests in Bonn

USE_DIFFICULT_OBJECTS = True

if USE_DIFFICULT_OBJECTS:
    # add the older robotics hall dataset
    name = "difficult_objects"

    # The directory containing the dataset to import
    data_dir = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/Bonn/difficult_objects_ais_coco"
    labels_path = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/Bonn/difficult_objects_ais_coco/annotations"

    train_dir = data_dir + '/train'
    valid_dir = data_dir + '/val'
    train_labels_path = labels_path + '/instances_Train.json'
    valid_labels_path = labels_path + '/instances_Validation.json'

    dataset_type = fo.types.COCODetectionDataset  # for example

    train_dataset_add = fo.Dataset.from_dir(
        dataset_type=dataset_type,
        data_path=train_dir,
        labels_path=train_labels_path,
        name=name+'_train',
    )

    valid_dataset_add = fo.Dataset.from_dir(
        dataset_type=dataset_type,
        data_path=valid_dir,
        labels_path=valid_labels_path,
        name=name+'_valid',
    )

    print('Adding dataset ' + name + '...')
    print('Found train dataset labels:')
    print(train_dataset_add.distinct("segmentations.detections.label"))
    print('Found valid dataset labels:')
    print(valid_dataset_add.distinct("segmentations.detections.label"))

    # most of the time we use all classes here and filter the total dataset at the end instead
    relevant_classes = ['apple', 'bag', 'banana', 'baseball', 'bowl', 'cereals',
                        'chair', 'chocolate_pudding_box', 'cleanser_bottle', 'coffee_can', 'cracker_box', 'dice', 
                        'dishwasher_tab', 'fork', 'knife', 'lemon', 'milk_bottle', 'mug', 'mustard_bottle', 
                        'orange', 'orange_juice', 'peach', 'pear', 'person', 
                        'plate', 'plum', 'potted_meat_can', 'pringles', 'rubiks_cube', 'soccer_ball', 
                        'spoon', 'strawberry', 'strawberry_gelatin_box', 'sugar_box', 'table', 
                        'tennis_ball', 'tomato_soup_can', 'tuna_fish_can']

    train_dataset_add = select_classes(train_dataset_add, labels=relevant_classes)
    valid_dataset_add = select_classes(valid_dataset_add, labels=relevant_classes)

    # note that we freeze the validation dataset
    combined_train_dataset = concat_datasets([combined_train_dataset, train_dataset_add])
    combined_train_dataset = concat_datasets([combined_train_dataset, valid_dataset_add])

 100% |█████████████████| 132/132 [2.7s elapsed, 0s remaining, 54.1 samples/s]      
 100% |███████████████████| 12/12 [250.5ms elapsed, 0s remaining, 48.2 samples/s]     
Adding dataset difficult_objects...
Found train dataset labels:
['almdudler', 'apple', 'apple_juice', 'bowl', 'cereals', 'chair', 'coffee_can', 'coke', 'cracker_box', 'fork', 'iso_drink', 'knife', 'lemon', 'mug', 'peach', 'pear', 'plate', 'pringles', 'spoon', 'strawberry', 'strawberry_gelatin_box', 'sugar_box', 'table', 'tennis_ball', 'tomato_soup_can']
Found valid dataset labels:
['almdudler', 'apple_juice', 'bowl', 'cereals', 'coffee_can', 'coke', 'cracker_box', 'fork', 'iso_drink', 'knife', 'lemon', 'mug', 'mustard_bottle', 'peach', 'pear', 'spoon', 'strawberry', 'strawberry_gelatin_box', 'sugar_box', 'tomato_soup_can']


In [8]:
# this is an older dataset, but its quite large and contains some useful objects
# it does not contribute to the validation set

USE_OLD_DATASET = True
if USE_OLD_DATASET:
    # add the older robotics hall dataset
    name = "hall_manyobject"

    # The directory containing the dataset to import
    data_dir = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/Bonn/base_dataset_ais_coco"
    labels_path = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/Bonn/base_dataset_ais_coco/annotations"

    train_dir = data_dir + '/train'
    valid_dir = data_dir + '/val'
    train_labels_path = labels_path + '/instances_Train.json'
    valid_labels_path = labels_path + '/instances_Validation.json'

    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset  # for example

    train_dataset_add = fo.Dataset.from_dir(
        dataset_type=dataset_type,
        data_path=train_dir,
        labels_path=train_labels_path,
        name=name+'_train',
    )

    valid_dataset_add = fo.Dataset.from_dir(
        dataset_type=dataset_type,
        data_path=valid_dir,
        labels_path=valid_labels_path,
        name=name+'_valid',
    )

    print('Adding dataset ' + name + '...')
    print('Found train dataset labels:')
    print(train_dataset_add.distinct("segmentations.detections.label"))
    print('Found valid dataset labels:')
    print(valid_dataset_add.distinct("segmentations.detections.label"))

    # most of the time we use all classes here and filter the total dataset at the end instead
    relevant_classes = ['bag', 'bowl', 'cereals', 'chair', 'cup', 'dishwasher_tab', 'fork', 
                        'knife', 'orange_juice', 'person', 'plate', 'pringles', 'spoon', 'table', 
                        'tomato_can', 'kidney_bean_can']

    train_dataset_add = select_classes(train_dataset_add, labels=relevant_classes)
    valid_dataset_add = select_classes(valid_dataset_add, labels=relevant_classes)
    
    # here tomato_can and kidney_bean_can are merged to tomato_soup_can
    # the false-positive risk is low because tomato_soup was the only can relevant in Bordeaux
    old_class_labels = ['orange_juice', 'cup', ['tomato_can', 'kidney_bean_can']]
    new_class_labels = ['tropical_juice', 'mug', 'tomato_soup_can']
    train_dataset_add = merge_to_superclass(train_dataset_add, new_class_labels, old_class_labels)
    valid_dataset_add = merge_to_superclass(valid_dataset_add, new_class_labels, old_class_labels)

    combined_train_dataset = concat_datasets([combined_train_dataset, train_dataset_add])
    combined_train_dataset = concat_datasets([combined_train_dataset, valid_dataset_add])

 100% |█████████████████| 507/507 [10.5s elapsed, 0s remaining, 57.9 samples/s]      
 100% |███████████████████| 30/30 [1.1s elapsed, 0s remaining, 28.5 samples/s]         
Adding dataset hall_manyobject...
Found train dataset labels:
['almdudler', 'apple_juice', 'bag', 'bell_pepper', 'booster', 'bowl', 'brush', 'cereals', 'chair', 'chamomile_tea', 'chocolate_candy', 'coke', 'cookies', 'couch', 'cup', 'dishwasher_tab', 'fork', 'iso_drink', 'ketchup', 'kidney_bean_can', 'kleenex', 'knife', 'mustard', 'noodles_chicken_flavor', 'noodles_duck_flavor', 'orange_juice', 'person', 'plate', 'pringles', 'sparkling_water', 'sponge', 'spoon', 'table', 'tomato_can', 'tuc', 'windex', 'wipes']
Found valid dataset labels:
['almdudler', 'apple_juice', 'bag', 'bell_pepper', 'booster', 'bowl', 'brush', 'cereals', 'chair', 'chamomile_tea', 'chocolate_candy', 'coke', 'cookies', 'couch', 'cup', 'dishwasher_tab', 'fork', 'iso_drink', 'ketchup', 'kidney_bean_can', 'kleenex', 'knife', 'mustard', 'noodles_chic

In [9]:
# add some auxiliary datasets from image searches here
# datasets are loaded, their labels merged and then filtered as defined below
# they also do not contribute to validation

aux_dataset_names = ['fruits', 'balls', 'cereals']
merge_superclasses = [None, None, None]
merge_labels = [None, None, None]
use_classes = [['apple', 'lemon', 'orange', 'peach', 'pear', 'person', 'plum', 'strawberry'], 
               ['soccer_ball', 'baseball', 'tennis_ball', 'person'], 
               ['cereals', 'strawberry']]

COMBINE = True
if COMBINE:
    ### load more datasets that are relevant
    for i, dataset_name in enumerate(aux_dataset_names):
        # The directory containing the dataset to import
        data_dir_add = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/image_search/" + dataset_name
        labels_path_add = "data/robocup_bordeaux_2023/robocup_data/coco_datasets/image_search/" + dataset_name + "/annotations"

        train_dir_add = data_dir_add + '/train'
        valid_dir_add = data_dir_add + '/val'
        train_labels_path_add = labels_path_add + '/instances_Train.json'
        valid_labels_path_add = labels_path_add + '/instances_Validation.json'

        train_dataset_add = fo.Dataset.from_dir(
            dataset_type=dataset_type,
            data_path=train_dir_add,
            labels_path=train_labels_path_add,
            name=dataset_name+'_train',
        )

        valid_dataset_add = fo.Dataset.from_dir(
            dataset_type=dataset_type,
            data_path=valid_dir_add,
            labels_path=valid_labels_path_add,
            name=dataset_name+'_valid',
        )
        
        # summarize dataset being added
        train_dataset_add_labels = train_dataset_add.distinct("segmentations.detections.label")
        valid_dataset_add_labels = valid_dataset_add.distinct("segmentations.detections.label")
        print('Adding dataset ' + dataset_name + '...')
        print('Training labels:', train_dataset_add_labels)
        print('Validation labels:', valid_dataset_add_labels)
        print()
        
        # merge some classes as specified if necessary
        train_dataset_add = merge_to_superclass(train_dataset_add, merge_superclasses[i], merge_labels[i])
        valid_dataset_add = merge_to_superclass(valid_dataset_add, merge_superclasses[i], merge_labels[i])
        # use only the classes we specified
        train_dataset_add = select_classes(train_dataset_add, labels=use_classes[i])
        valid_dataset_add = select_classes(valid_dataset_add, labels=use_classes[i])

        # combine these with the previous dataset
        combined_train_dataset = concat_datasets([combined_train_dataset, train_dataset_add])
        combined_train_dataset = concat_datasets([combined_train_dataset, valid_dataset_add])

 100% |███████████████████| 88/88 [2.8s elapsed, 0s remaining, 33.2 samples/s]        
 100% |███████████████████| 10/10 [156.7ms elapsed, 0s remaining, 63.8 samples/s]     
Adding dataset fruits...
Training labels: ['apple', 'bag', 'banana', 'bell_pepper', 'chair', 'dishwasher', 'lemon', 'orange', 'orange_juice', 'peach', 'pear', 'person', 'plum', 'schweppes_lemon', 'strawberry', 'table']
Validation labels: ['apple', 'banana', 'bell_pepper', 'chair', 'lemon', 'orange', 'orange_juice', 'pear', 'plum']

 100% |███████████████████| 89/89 [1.3s elapsed, 0s remaining, 66.3 samples/s]         
 100% |███████████████████| 11/11 [89.7ms elapsed, 0s remaining, 122.6 samples/s]     
Adding dataset balls...
Training labels: ['baseball', 'golf_ball', 'person', 'racquetball', 'soccer_ball', 'soft_ball', 'tennis_ball']
Validation labels: ['baseball', 'golf_ball', 'soccer_ball', 'tennis_ball']

 100% |███████████████████| 88/88 [1.2s elapsed, 0s remaining, 70.7 samples/s]          
 100% |██████████

In [10]:
# in this cell we can further customize the dataset which we have constructed above
# usually this means merging or filtering labels in a way which concerns all contributing datasets

CUSTOMIZE = True
if CUSTOMIZE:
    # this renaming step ensures that we use the official class labels at RoboCup@Home23
    old_class_labels = ['chocolate_pudding_box', 'cleanser_bottle', 'coffee_can', 'cracker_box', 'jacket_stand', 'mug', 'mustard_bottle', 'potted_meat_can', 'strawberry_gelatin_box', 
                        'sugar_box', 'tomato_soup_can', 'tuna_fish_can', 'milk_bottle']
    new_class_labels = ['chocolate_jello', 'cleanser', 'coffee_grounds', 'cheezit', 'clothes_rack', 'cup', 'mustard', 'spam', 'strawberry_jello', 
                        'sugar', 'tomato_soup', 'tuna', 'milk']
    train_dataset_processed = merge_to_superclass(combined_train_dataset, new_class_labels, old_class_labels)
    valid_dataset_processed = merge_to_superclass(combined_valid_dataset, new_class_labels, old_class_labels)
    
    # finally constrain to classes relevant for task
    task_specific_classes = ['apple', 'banana', 'baseball', 'cereals', 'chocolate_jello', 'cleanser', 'coffee_grounds', 'cola', 
                        'couch_table', 'cheezit', 'dice', 'fork', 'iced_tea', 'juice_pack', 'knife', 'lemon', 'milk', 
                        'peach', 'mustard', 'orange', 'orange_juice', 'pear', 'plum', 'spam', 'pringles', 'rubiks_cube', 
                        'soccer_ball', 'sponge', 'spoon', 'strawberry', 'strawberry_jello', 'sugar', 'tennis_ball', 
                        'tomato_soup', 'tropical_juice', 'tuna', 'shelf', 'shelf_door', 'red_wine']

    train_dataset_processed = select_classes(train_dataset_processed, labels=task_specific_classes)
    valid_dataset_processed = select_classes(valid_dataset_processed, labels=task_specific_classes)
else:
    train_dataset_processed = combined_train_dataset
    valid_dataset_processed = combined_valid_dataset

In [11]:
# this cell bridges from fiftyone to detectron2 and allows sanity-checking the constructed dataset
classes = train_dataset_processed.distinct("segmentations.detections.label")
labels_dict = {class_label: class_idx for class_idx, class_label in enumerate(classes)}
print('Label dictionary for complete dataset')
print(labels_dict)

# dump labels for deployment
os.makedirs('output', exist_ok=True)
with open("output/classes.json", "w") as fp:
    json.dump(classes, fp)

# bridge fiftyone to detectron2
for dataset, tag in [(train_dataset_processed, "train"), (valid_dataset_processed, "valid")]:
    view = dataset
    if "fiftyone_" + tag in DatasetCatalog.list():
        DatasetCatalog.remove("fiftyone_" + tag)
    DatasetCatalog.register("fiftyone_" + tag, lambda view=view: get_fiftyone_dicts(view, labels_dict))
    MetadataCatalog.get("fiftyone_" + tag).set(thing_classes=classes, evaluator_type="coco")
metadata = MetadataCatalog.get("fiftyone_train")

Label dictionary for complete dataset
{'apple': 0, 'banana': 1, 'baseball': 2, 'cereals': 3, 'cheezit': 4, 'chocolate_jello': 5, 'cleanser': 6, 'coffee_grounds': 7, 'cola': 8, 'couch_table': 9, 'dice': 10, 'fork': 11, 'iced_tea': 12, 'juice_pack': 13, 'knife': 14, 'lemon': 15, 'milk': 16, 'mustard': 17, 'orange': 18, 'orange_juice': 19, 'peach': 20, 'pear': 21, 'plum': 22, 'pringles': 23, 'red_wine': 24, 'rubiks_cube': 25, 'shelf': 26, 'shelf_door': 27, 'soccer_ball': 28, 'spam': 29, 'sponge': 30, 'spoon': 31, 'strawberry': 32, 'strawberry_jello': 33, 'sugar': 34, 'tennis_ball': 35, 'tomato_soup': 36, 'tropical_juice': 37, 'tuna': 38}


#### Detector Training

Now that the task-specific dataset is ready, we can finetune a pretrained MaskDINO model on the selected data.

In [12]:
# setup and launch the trainer
from helpers.model_trainer import Trainer
import maskdino
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

# in case our dataset was changed discard previous cache
try:
    for item in os.listdir("output/inference"):
        os.remove("output/inference/" + item)
except:
    pass

cfg = get_cfg()
add_deeplab_config(cfg)
maskdino.add_maskdino_config(cfg)
if model_type == 'swin':
    cfg.merge_from_file('data/robocup_bordeaux_2023/robocup_data/trained_models/conf/swin_config.yaml')
    cfg.MODEL.WEIGHTS = "data/robocup_bordeaux_2023/robocup_data/trained_models/coco_pretrained_maskdino/swin/maskdino_swinl_50ep_300q_hid2048_3sd1_instance_maskenhanced_mask52.3ap_box59.0ap.pth" 
    cfg.SOLVER.IMS_PER_BATCH = 4 # batch size
elif model_type == 'r50':
    cfg.merge_from_file('data/robocup_bordeaux_2023/robocup_data/trained_models/conf/r50_config.yaml')
    cfg.MODEL.WEIGHTS = "data/robocup_bordeaux_2023/robocup_data/trained_models/coco_pretrained_maskdino/resnet/maskdino_r50_50ep_300q_hid2048_3sd1_instance_maskenhanced_mask46.3ap_box51.7ap.pth"
    cfg.SOLVER.IMS_PER_BATCH = 4
cfg.DATASETS.TRAIN = ('fiftyone_train',)
cfg.DATASETS.TEST = ('fiftyone_valid',)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = len(classes)
cfg.SOLVER.MAX_ITER = 2400  # corresponds to about 30 minutes of finetuning on an RTX4090
cfg.SOLVER.STEPS = (800, 1400, 2000) # when to decay learning rate

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
TRAIN = True
if TRAIN:
    trainer = Trainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

/home/user/nogga/anaconda3/envs/athome23_detection/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_interm_dn_0': 2.0, 'loss_ce_1': 4.0, 'loss_mask_1':

[07/13 11:11:29 d2.data.build]: Removed 1 images with no usable annotations. 1512 images left.
[07/13 11:11:29 d2.data.build]: Distribution of instances among all 39 categories:
|   category    | #instances   |   category    | #instances   |   category    | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
|     apple     | 417          |    banana     | 139          |   baseball    | 234          |
|    cereals    | 1071         |    cheezit    | 258          | chocolate_j.. | 194          |
|   cleanser    | 178          | coffee_grou.. | 311          |     cola      | 144          |
|  couch_table  | 147          |     dice      | 73           |     fork      | 397          |
|   iced_tea    | 146          |  juice_pack   | 149          |     knife     | 390          |
|     lemon     | 336          |     milk      | 241          |    mustard    | 162          |
|    orange     | 277          | orange_juice  | 112          

Skip loading parameter 'sem_seg_head.predictor.class_embed.weight' to the model due to incompatible shapes: (80, 256) in the checkpoint but (39, 256) in the model! You might want to double check if this is expected.
Skip loading parameter 'sem_seg_head.predictor.class_embed.bias' to the model due to incompatible shapes: (80,) in the checkpoint but (39,) in the model! You might want to double check if this is expected.
Skip loading parameter 'sem_seg_head.predictor.label_enc.weight' to the model due to incompatible shapes: (80, 256) in the checkpoint but (39, 256) in the model! You might want to double check if this is expected.
Skip loading parameter 'criterion.empty_weight' to the model due to incompatible shapes: (81,) in the checkpoint but (40,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
criterion.empty_weight
sem_seg_head.predictor.class_embed.{bias, weight}
sem_seg_head.predictor.label_enc.wei

[07/13 11:11:29 d2.engine.train_loop]: Starting training from iteration 0


...

[07/13 11:40:44 d2.utils.events]:  eta: 0:03:08  iter: 2159  total_loss: 31.07  loss_ce: 0.5557  loss_mask: 0.04721  loss_dice: 0.3548  loss_bbox: 0.04642  loss_giou: 0.2185  loss_ce_dn: 0.146  loss_mask_dn: 0.03953  loss_dice_dn: 0.399  loss_bbox_dn: 0.03131  loss_giou_dn: 0.1585  loss_ce_0: 0.9197  loss_mask_0: 0.05283  loss_dice_0: 0.4478  loss_bbox_0: 0.0576  loss_giou_0: 0.3352  loss_ce_dn_0: 1.753  loss_mask_dn_0: 0.5189  loss_dice_dn_0: 2.625  loss_bbox_dn_0: 0.3341  loss_giou_dn_0: 0.8559  loss_ce_1: 0.8172  loss_mask_1: 0.04773  loss_dice_1: 0.4306  loss_bbox_1: 0.04938  loss_giou_1: 0.2667  loss_ce_dn_1: 0.2787  loss_mask_dn_1: 0.04339  loss_dice_dn_1: 0.4685  loss_bbox_dn_1: 0.073  loss_giou_dn_1: 0.2742  loss_ce_2: 0.7154  loss_mask_2: 0.04675  loss_dice_2: 0.4357  loss_bbox_2: 0.05106  loss_giou_2: 0.2572  loss_ce_dn_2: 0.2174  loss_mask_dn_2: 0.0453  loss_dice_dn_2: 0.424  loss_bbox_dn_2: 0.04673  loss_giou_dn_2: 0.199  loss_ce_3: 0.6437  loss_mask_3: 0.04395  loss_dice_3

[07/13 11:41:47 d2.utils.events]:  eta: 0:02:05  iter: 2239  total_loss: 29.46  loss_ce: 0.5051  loss_mask: 0.05084  loss_dice: 0.3832  loss_bbox: 0.03742  loss_giou: 0.2049  loss_ce_dn: 0.1143  loss_mask_dn: 0.0432  loss_dice_dn: 0.3901  loss_bbox_dn: 0.02567  loss_giou_dn: 0.1624  loss_ce_0: 0.9104  loss_mask_0: 0.04914  loss_dice_0: 0.4205  loss_bbox_0: 0.06299  loss_giou_0: 0.3372  loss_ce_dn_0: 1.874  loss_mask_dn_0: 0.474  loss_dice_dn_0: 2.768  loss_bbox_dn_0: 0.3176  loss_giou_dn_0: 0.8564  loss_ce_1: 0.784  loss_mask_1: 0.04801  loss_dice_1: 0.4218  loss_bbox_1: 0.04958  loss_giou_1: 0.2401  loss_ce_dn_1: 0.2708  loss_mask_dn_1: 0.05419  loss_dice_dn_1: 0.4177  loss_bbox_dn_1: 0.07139  loss_giou_dn_1: 0.2676  loss_ce_2: 0.6458  loss_mask_2: 0.04449  loss_dice_2: 0.3963  loss_bbox_2: 0.04393  loss_giou_2: 0.2185  loss_ce_dn_2: 0.1756  loss_mask_dn_2: 0.04687  loss_dice_dn_2: 0.3944  loss_bbox_dn_2: 0.04166  loss_giou_dn_2: 0.1952  loss_ce_3: 0.5827  loss_mask_3: 0.05264  loss_d

[07/13 11:42:51 d2.utils.events]:  eta: 0:01:02  iter: 2319  total_loss: 30.72  loss_ce: 0.5373  loss_mask: 0.03038  loss_dice: 0.4391  loss_bbox: 0.04873  loss_giou: 0.2592  loss_ce_dn: 0.125  loss_mask_dn: 0.02873  loss_dice_dn: 0.4049  loss_bbox_dn: 0.02482  loss_giou_dn: 0.1723  loss_ce_0: 0.9794  loss_mask_0: 0.02902  loss_dice_0: 0.4785  loss_bbox_0: 0.06858  loss_giou_0: 0.4207  loss_ce_dn_0: 1.931  loss_mask_dn_0: 0.325  loss_dice_dn_0: 2.861  loss_bbox_dn_0: 0.2517  loss_giou_dn_0: 0.8578  loss_ce_1: 0.8569  loss_mask_1: 0.0313  loss_dice_1: 0.4741  loss_bbox_1: 0.05551  loss_giou_1: 0.2961  loss_ce_dn_1: 0.2691  loss_mask_dn_1: 0.02982  loss_dice_dn_1: 0.4245  loss_bbox_dn_1: 0.05456  loss_giou_dn_1: 0.2936  loss_ce_2: 0.7387  loss_mask_2: 0.03088  loss_dice_2: 0.4312  loss_bbox_2: 0.05365  loss_giou_2: 0.2816  loss_ce_dn_2: 0.19  loss_mask_dn_2: 0.02944  loss_dice_dn_2: 0.405  loss_bbox_dn_2: 0.03781  loss_giou_dn_2: 0.2202  loss_ce_3: 0.6518  loss_mask_3: 0.03087  loss_dice

[07/13 11:44:01 d2.utils.events]:  eta: 0:00:00  iter: 2399  total_loss: 25.16  loss_ce: 0.4077  loss_mask: 0.05206  loss_dice: 0.3056  loss_bbox: 0.04354  loss_giou: 0.1606  loss_ce_dn: 0.07873  loss_mask_dn: 0.05005  loss_dice_dn: 0.3178  loss_bbox_dn: 0.03255  loss_giou_dn: 0.1447  loss_ce_0: 0.8513  loss_mask_0: 0.05454  loss_dice_0: 0.3151  loss_bbox_0: 0.0685  loss_giou_0: 0.301  loss_ce_dn_0: 1.843  loss_mask_dn_0: 0.5186  loss_dice_dn_0: 2.667  loss_bbox_dn_0: 0.4041  loss_giou_dn_0: 0.8579  loss_ce_1: 0.7317  loss_mask_1: 0.05406  loss_dice_1: 0.3176  loss_bbox_1: 0.05487  loss_giou_1: 0.2004  loss_ce_dn_1: 0.2595  loss_mask_dn_1: 0.05548  loss_dice_dn_1: 0.3079  loss_bbox_dn_1: 0.09274  loss_giou_dn_1: 0.2534  loss_ce_2: 0.5933  loss_mask_2: 0.04833  loss_dice_2: 0.2975  loss_bbox_2: 0.04899  loss_giou_2: 0.179  loss_ce_dn_2: 0.1685  loss_mask_dn_2: 0.05068  loss_dice_dn_2: 0.3149  loss_bbox_dn_2: 0.04847  loss_giou_dn_2: 0.186  loss_ce_3: 0.4951  loss_mask_3: 0.04786  loss_d

[07/13 11:44:19 d2.evaluation.coco_evaluation]: Per-category segm AP: 
| category         | AP     | category       | AP     | category        | AP     |
|:-----------------|:-------|:---------------|:-------|:----------------|:-------|
| apple            | 86.040 | banana         | 88.995 | baseball        | 72.414 |
| cereals          | 79.675 | cheezit        | 84.653 | chocolate_jello | 75.856 |
| cleanser         | 70.902 | coffee_grounds | 79.213 | cola            | 80.089 |
| couch_table      | 49.020 | dice           | 64.219 | fork            | 70.438 |
| iced_tea         | 98.193 | juice_pack     | 84.684 | knife           | 71.218 |
| lemon            | 88.575 | milk           | 83.384 | mustard         | 86.197 |
| orange           | 67.089 | orange_juice   | 99.345 | peach           | 74.010 |
| pear             | 86.717 | plum           | 84.873 | pringles        | 86.802 |
| red_wine         | 89.916 | rubiks_cube    | 82.102 | shelf           | 76.834 |
| shelf_door    

#### Exporting the Detector

The detector is fully described by its model definition in the form of a detectron2 config, its weights after training and a file enumerating the object class labels used in the particular training session. These are gathered and zipped in the following.

In [13]:
# save the model in a format suitable for deployment on the robot
# this is just a zip file containing weights, class label descriptions and a model description
if TRAIN:
    model_path = 'output/model_final.pth'
    conf_path = 'data/robocup_bordeaux_2023/robocup_data/trained_models/conf/swin_config.yaml' if model_type == 'swin' else 'data/robocup_bordeaux_2023/robocup_data/trained_models/conf/r50_config.yaml'
    classes_path = 'output/classes.json'


    time_stamp = get_timestamp()
    result_path = 'output/' + experiment_name + '_' + time_stamp

    try:
        shutil.rmtree(result_path)
    except:
        pass

    os.mkdir(result_path)

    print('Copy result files...')
    shutil.copy(model_path, result_path + '/model.pth')
    shutil.copy(conf_path, result_path + '/detectron2_config.yaml') 
    shutil.copy(classes_path, result_path + '/class_labels.json') 
    print('ZIP result files...')
    make_archive(result_path, result_path+'.zip')
    print('Done!')

Copy result files...
ZIP result files...
Done!


#### Evaluating the Detector

To make sure that the detector is well equipped to find objects relevant to the task at hand, its outputs on the validation set are brought back from detectron2 to fiftyone for visualization. In addition, precision, recall and f1-scores are computed class-wise to help identify any issues.

In [14]:
# if we just want to load the results of a previous model use that models experiment_name here
# else leave model_name at None to use the most recently trained model
model_name = None
weight_path = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") if model_name is None else os.path.join(cfg.OUTPUT_DIR, model_name, 'model.pth')
cfg.MODEL.WEIGHTS =  weight_path
predictor = DefaultPredictor(cfg)

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_interm_dn_0': 2.0, 'loss_ce_1': 4.0, 'loss_mask_1':

In [15]:
# bridge back from detectron2 to fiftyone to annotate the validation data with model predictions

val_view = valid_dataset_processed
dataset_dicts = get_fiftyone_dicts(val_view, labels_dict)
predictions = {}
for d in dataset_dicts:
    img_w = d["width"]
    img_h = d["height"]
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    detections, instances = detectron_to_fo(outputs, img_w, img_h, classes)
    predictions[d["image_id"]] = detections

valid_dataset_processed.set_values("predictions", predictions, key_field="id")

In [16]:
# launch a fiftyone session to view the model predictions after training
session = fo.launch_app(valid_dataset_processed)

In [17]:
# this is just a quick summary of model performance as a final sanity-check before deployment
# in addition, the training process above yields validation map scores as a side result
results = valid_dataset_processed.evaluate_detections(
    "predictions",
    gt_field="segmentations",
    eval_key="eval",
    use_masks=True,
    compute_mAP=True,
)
results.print_report()

Evaluating detections...
 100% |███████████████████| 58/58 [4.3s elapsed, 0s remaining, 17.7 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 58/58 [3.3s elapsed, 0s remaining, 23.5 samples/s]      
                  precision    recall  f1-score   support

           apple       0.85      0.85      0.85        13
          banana       1.00      0.91      0.95        11
        baseball       0.58      0.70      0.64        10
         cereals       0.73      0.79      0.76        34
         cheezit       1.00      0.70      0.82        10
 chocolate_jello       0.80      0.92      0.86        13
        cleanser       0.91      0.83      0.87        12
  coffee_grounds       0.80      1.00      0.89         8
            cola       0.79      1.00      0.88        15
     couch_table       1.00      0.60      0.75         5
            dice       1.00      1.00      1.00         7
            fork       0.64      0.70      0.67        10
        iced_tea       1.0